# PyTorchのトレーニング♨

## 環境準備、共通的データ処理 編

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
  - [インストール](#インストール)
  - [インポート](#インポート)
  - [プロキシ設定](#プロキシ設定)
- [共通処理](#共通処理)
  - [勾配降下法アルゴリズムの選択](#勾配降下法アルゴリズムの選択)
  - [モデルの定義とコンパイル](#モデルの定義とコンパイル)
  - [モデルの学習と推論](#モデルの学習と推論モデルの保存と復元)
  - [モデルの保存と復元](#モデルの保存と復元)
  - [過学習の抑止](#過学習の抑止)

## 環境準備

### インストール

#### 基本的なライブラリ

In [ ]:
!pip install numpy
!pip install pandas
!pip install seaborn
!pip install statsmodels
!pip install tqdm # プログレスバー（学習関数で使用する

#### scikit-learn

In [ ]:
!pip install scikit-learn

#### PyTorch

In [ ]:
! pip install torch
! pip install torchvision

#### opencv

In [ ]:
!pip install opencv-python

### インポート

#### 基本的なライブラリ

In [ ]:
import io
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### scikit-learn

In [ ]:
from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

#### PyTorch

In [ ]:
import torch
import torchvision
print("torch ver : " + torch.__version__)
print("torchvision ver : " + torchvision.__version__)

In [ ]:
#モデル定義ライブラリ
import torch.nn as nn
import torch.nn.functional as F

# 最適化関数ライブラリ
import torch.optim as optim

#### その他

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### プロキシ設定

In [ ]:
import os
proxies = {
    "http": os.getenv("HTTP_PROXY"),
    "https": os.getenv("HTTPS_PROXY")
}

## 共通処理

### torch.Tensor

#### numpy.ndarrayとtorch.Tensor

##### numpy.ndarrayとtorch.Tensorの生成

In [ ]:
# numpy.ndarrayの作成
numpy_array = np.array([0, 1, 2])
numpy_array

In [ ]:
# torch.Tensorの作成
torch_tensor = torch.tensor([0, 1, 2])
torch_tensor

##### numpy.ndarrayとtorch.Tensorの相互変換

In [ ]:
# torch → numpy変換
torch_array = torch.from_numpy(numpy_array)
torch_array

In [ ]:
numpy_array = torch_array.numpy()
numpy_array

#### torch.Tensorの操作
torch.view()とtorch.transpose()

##### torch.view()
ndarray.reshape的な

In [ ]:
# 2行3列のtorch.Tensorの作成
torch_array = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(torch_array)
print(torch_array.size())

In [ ]:
# 3行2列に形状変化（reshape的な）
torch_array = torch_array.view(3, 2)
print(torch_array)
print(torch_array.size())

In [ ]:
# ｎ行３列に形状変化（ｎは自動計算）
torch_array = torch_array.view(-1, 3)
print(torch_array)
print(torch_array.size())

##### torch.transpose()
ndarray.transpose的な

In [ ]:
# 3行3列のtorch.tensorの作成
torch_array = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]]) 
print(torch_array)
print(torch_array.size())

In [ ]:
# 転置
torch_array = torch_array.transpose(0, 1) #２軸を指定可
print(torch_array)
print(torch_array.size())

In [ ]:
# 転置
torch_array = torch_array.t() # 行列時の略記法
print(torch_array)

### モデルの定義とコンパイル
以下は、DNNによる回帰

#### モデルの定義

##### torch.nn.Moduleのサブクラスを定義してモデルを構築
一方通行ではない複雑なモデル（ネットワーク）の場合

```python
class Model(nn.Module):
    # 各層を__init__に定義する
    def __init__(self):
        # 親クラスの継承
        super(Model, self).__init__()
        
        # 全結合層の定義
        # nn.Linear(入力層の数, 出力層の数)
        self.fc1 = nn.Linear(784, 900)
        self.fc2 = nn.Linear(900, 1000)
        self.fc3 = nn.Linear(1000, 500)
        self.output = nn.Linear(500, 10)
        
    # 順伝搬の計算の定義
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        
        return F.relu(self.output(x))
```

##### torch.nn.Sequentialでモデルを構築
一方通行のシンプルなモデル（ネットワーク）の場合

In [ ]:
class Model(nn.Module):
    # 各層を__init__に定義する
    def __init__(self):
        # 親クラスの継承
        super(Model, self).__init__()
        
        # nn.Sequentalで処理をまとめる
        self.layers = nn.Sequential(
            # nn.Linear(入力層の数, 出力層の数)
            nn.Linear(13, 64),
            # inplace=Trueを指定することでメモリを節約できる(推奨)
            nn.ReLU(inplace=True),
            
            nn.Linear(64, 1),
            nn.ReLU(inplace=True)
        )
        
    # 順伝搬の計算の定義
    def forward(self, x):
        return self.layers(x)

#### モデルのコンパイル

In [ ]:
# ネットワークの呼び出し
model = Model()

### 勾配降下法アルゴリズムの選択
PyTorchではmodel定義がないと動かない。  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%81%AE%E3%83%86%E3%82%AF%E3%83%8B%E3%83%83%E3%82%AF#ma318ba7

#### SGD
詳しくは[コチラ](TensorFlowAndKeras0.ipynb)

In [ ]:
# lr: 学習率，momentum: 慣性項 
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) 

#### MomentumSGD
詳しくは[コチラ](TensorFlowAndKeras0.ipynb)

In [ ]:
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#### NesterovAG 
詳しくは[コチラ](TensorFlowAndKeras0.ipynb)

In [ ]:
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)

#### Adagrad
詳しくは[コチラ](TensorFlowAndKeras0.ipynb)

In [ ]:
#optimizer = optim.Adagrad(model.parameters(), lr=0.01, eps=1e-10)

#### RMSprop
詳しくは[コチラ](TensorFlowAndKeras0.ipynb)

In [ ]:
#optimizer = optim.RMSprop(model.parameters(), lr=0.01, alpha=0.99, eps=1e-08)

#### Adadelta
詳しくは[コチラ](TensorFlowAndKeras0.ipynb)

In [ ]:
#optimizer = optim.Adadelta(model.parameters(),lr=1.0, rho=0.9, eps=1e-06)

#### Adam
詳しくは[コチラ](TensorFlowAndKeras0.ipynb)

In [ ]:
#optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)

### モデルの定義とコンパイル
一先ず[PyTorch3.ipynb](PyTorch3.ipynb)を参照。

### モデルの学習と推論
一先ず[PyTorch3.ipynb](PyTorch3.ipynb)を参照。

### モデルの保存と復元
一先ず[PyTorch3.ipynb](PyTorch3.ipynb)を参照。

### 過学習の抑止
一先ず[PyTorch3.ipynb](PyTorch3.ipynb)を参照。